In [27]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyzing a GEOGRAPHY column with `bigframes.geopandas.GeoSeries`

In [28]:
import bigframes
import bigframes.geopandas
import bigframes.pandas as bpd
import shapely
bpd.options.display.progress_bar = None

### Load the Counties table from the Census Bureau US Boundaries dataset

In [29]:
df = bpd.read_gbq("bigquery-public-data.geo_us_boundaries.counties")

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: UserWarning: Reading cached table from 2025-02-03 22:54:14.603760+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/session/_io/bigquery/read_gbq_table.py:274: DefaultIndexWarning: Table 'bigquery-public-data.geo_us_boundaries.counties' is clustered and/or partitioned, but BigQuery DataFrames was not able to find a suitable index. To avoid this warning, set at least one of: `index_col` or `filters`.
  warnings.warn(msg, category=bfe.DefaultIndexWarning)


### Create a series from the int_point_geom column

In [30]:
point_geom_series = df['int_point_geom']

## The `GeoSeries` constructor accepts local data or a `bigframes.pandas.Series` object.

### 1. Create a GeoSeries from local data with `Peek`

In [31]:
five_geo_points = point_geom_series.peek(n = 5)
five_geo_points

7      POINT (-82.42267 38.14364)
34     POINT (-89.99324 34.87427)
87     POINT (-81.86504 29.98658)
234    POINT (-97.29122 32.77212)
280    POINT (-94.53725 43.67712)
Name: int_point_geom, dtype: geometry

### Convert the five geo points to `bigframes.gopandas.GeoSeries`

#### Note: TypeError is raised if the GEOGRAPHY column contains geometry type other than `Point`.

In [32]:
geo_points = bigframes.geopandas.GeoSeries(
        [point for point in five_geo_points]
)
geo_points

0    POINT (-82.42267 38.14364)
1    POINT (-89.99324 34.87427)
2    POINT (-81.86504 29.98658)
3    POINT (-97.29122 32.77212)
4    POINT (-94.53725 43.67712)
dtype: geometry

### Retrieve the x (longitude) and y (latitude) from the GeoSeries with `.x` and `.y`.

### `.x`

In [33]:
geo_points.x

0   -82.422666
1    -89.99324
2   -81.865036
3   -97.291224
4   -94.537252
dtype: Float64

### `.y`

In [34]:
geo_points.y

0    38.143642
1    34.874266
2    29.986584
3    32.772119
4    43.677118
dtype: Float64

### 2. Alternatively, use the `.geo` accessor to access GeoSeries methods from a `bigframes.pandas.Series` object.

#### `geo.x`

In [35]:
point_geom_series.geo.x

0    -101.298265
1     -99.111085
2      -66.58687
3    -102.601791
4     -71.578625
5     -88.961529
6     -87.492986
7     -82.422666
8    -100.208166
9     -85.815939
10   -101.681133
11   -119.516659
12    -89.398306
13    -107.78848
14    -91.159306
15   -113.887042
16    -83.470416
17    -98.520146
18    -83.911718
19    -87.321865
20    -91.727626
21    -93.466093
22   -101.143324
23    -78.657634
24    -94.272323
dtype: Float64

#### `geo.y`

In [36]:
point_geom_series.geo.y

0     46.710819
1     29.353661
2     18.211152
3     38.835646
4     41.869768
5     39.860237
6     36.892059
7     38.143642
8     34.524623
9     30.862007
10    40.180165
11    46.228125
12    36.054196
13    38.154731
14    38.761902
15    44.928506
16    30.447232
17    29.448671
18    42.602532
19    34.529776
20    33.957675
21    42.037538
22    29.875285
23    36.299884
24    44.821657
dtype: Float64

## Retrive the `area` of different geometrty shapes. 

### 1. Create a gemometry collection from local data with `Peek`

In [37]:
geom_series = df["county_geom"].peek(n = 5)
geom_series

18     POLYGON ((-84.14483 42.59956, -84.14475 42.598...
86     POLYGON ((-90.3355 43.20969, -90.33658 43.2097...
177    POLYGON ((-117.49995 48.65277, -117.49995 48.6...
208    POLYGON ((-84.69253 36.43547, -84.69244 36.435...
300    POLYGON ((-92.08001 43.42885, -92.08 43.4277, ...
Name: county_geom, dtype: geometry

### Convert the geometry collection to `bigframes.gopandas.GeoSeries`

In [38]:
five_geom = bigframes.geopandas.GeoSeries(
        [point for point in geom_series]
)
five_geom

0    POLYGON ((-84.14483 42.59956, -84.14475 42.598...
1    POLYGON ((-90.3355 43.20969, -90.33658 43.2097...
2    POLYGON ((-117.49995 48.65277, -117.49995 48.6...
3    POLYGON ((-84.69253 36.43547, -84.69244 36.435...
4    POLYGON ((-92.08001 43.42885, -92.08 43.4277, ...
dtype: geometry

## Note: `bigframes.geopandas.GeoSeries.area` raises NotImplementedError.  

In [39]:
five_geom.area

NotImplementedError: GeoSeries.area is not supported. Use bigframes.bigquery.st_area(series), instead. Share your usecase with the BigQuery DataFrames team at the https://bit.ly/bigframes-feedback survey.You are currently running BigFrames version 1.34.0

## Use `bigframes.bigquery.st_area` to retirive the `area` in square meters instead. See: https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_area

In [40]:
import bigframes.bigquery as bbq

In [41]:
geom_area = bbq.st_area(five_geom)
geom_area

0    1513674358.195978
1    1985759505.347184
2    3680789945.308985
3    1380731457.626208
4    1784154727.752777
dtype: Float64